## PARTE I 

### Imports

In [ ]:
%%capture
from numpy import load
!pip install datasets #--Data
from datasets import load_dataset
import pandas as pd 
import seaborn as sns #--Data analysis 
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import STOPWORDS, WordCloud
!pip install ftfy
import ftfy # import libraries
import nltk
import json
import re
%matplotlib inline
import csv
import warnings
warnings.filterwarnings('ignore')
np.random.seed(0)

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV,cross_validate, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import Binarizer, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import tree
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier # need to import xboost calssifer

from collections import Counter, defaultdict
from os import listdir, makedirs
from os.path import isfile, join, splitext, split
from unicodedata import normalize
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from gensim.models import Word2Vec
#nltk.download('stopwords')
# stop words list set to english
stopwords_list = stopwords.words('english') # stop word list
import re
import string
!pip install zeugma
!pip install preprocessing
!pip install function
from zeugma.embeddings import EmbeddingTransformer
from preprocessing import *
from function import *
#!pip install itertools
import itertools
#!pip install bert_embedding
#from bert_embedding import BertEmbedding
#!pip install transformers
#import transformers
#from transformers import pipeline, AutoTokenizer

#classifiers
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
import tensorflow as tf#--cnn
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

#list(stopwords_list)

### Data 

In [ ]:
#dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/5_Data/FAKES/dados/FA-KES-Dataset.csv')
data.rename(columns = {'article_content':'statement','labels':'label'}, inplace = True)

In [ ]:
print(pd.isnull(data['statement']).sum())
print(pd.isnull(data['label']).sum())
print(pd.isnull(data).sum())

0
0
unit_id          0
article_title    0
statement        0
source           0
date             0
location         0
label            0
dtype: int64


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   unit_id        804 non-null    int64 
 1   article_title  804 non-null    object
 2   statement      804 non-null    object
 3   source         804 non-null    object
 4   date           804 non-null    object
 5   location       804 non-null    object
 6   label          804 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 44.1+ KB


In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=0)
train, val = train_test_split(train, test_size=0.2, random_state=0)

### Pre process for Statement 

This pre processing will be applied to all tecniques (CV, TFIDF, Glove, W2V, FAST) 

Dos pre processamentos, o multiclass tem desbalanceio na classe 5 , o sentimental liar é mais desbalanceado que o pré processamento 2. 

In [ ]:
#verificaçao de valores nulos - #without blank values
#print(pd.isnull(train['statement']).sum(),pd.isnull(test['statement']).sum(),pd.isnull(val['statement']).sum())
#print(pd.isnull(train['label']).sum(),pd.isnull(test['label']).sum(),pd.isnull(val['label']).sum())

In [ ]:
%%time
print(train['statement'][0])
print(train['statement'][7])
# fontes: analises pessoais + https://github.com/mansoor9743/Fake-News-Detection/blob/master/code/Fake_news_detection.ipynb 
#(com adaptaçoes porque foi adotado o LEM ao inves de STEM) + https://www.coursera.org/learn/classification-vector-spaces-in-nlp/supplement/SLqys/preprocessing
# stop words list set to english
stopwords_list = stopwords.words('english') # stop word list

def clean_text(text): 
  # etapas: decontrar, tirar caracteries especiais usando string.punctuation, tira url, tira IPADDRESS, tira pontuaçoes/numeros, tira stopwords com tamanho >=3, lower case
  #tokenizer, stemizaçao e remoçao de stopwords, detalhe que volta pro formato original, vendo se é preciso criar funçao de tokenizer 
  #alterei a ordem
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    text = re_url.sub("URL", text)
    text = re_ip.sub("IPADDRESS", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
#    text = " ".join(stemmed_words)
#    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in stemmed_words if not w in stops and len(w) >= 3]
    text = " ".join(text)
    return text

#--tokenizer #https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794
def tokenizer(text, flg_stemm=True, flg_lemm=False, lst_stopwords=stopwords_list):#None):
    lst_text = text.lower().split()#text.split()
  
    if flg_stemm == True: ## Stemming (remove -ing, -ly, ...)
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
   
    if flg_lemm == True: ## Lemmatisation (convert the word into root word)
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]  
  
    text = " ".join(lst_text)  ## back to string from list
    return text #let in original format to run with word embeddings based on native tokenizer

train['statement']=train['statement'].map(clean_text) #applying 1 
test['statement']=test['statement'].map(clean_text)
val['statement']=val['statement'].map(clean_text)

Wed 05 Apr 2017 Syria attack symptoms consistent with nerve agent use WHO. Victims of a suspected chemical attack in Syria appeared to show symptoms consistent with reaction to a nerve agent the World Health Organization said on Wednesday. "Some cases appear to show additional signs consistent with exposure to organophosphorus chemicals a category of chemicals that includes nerve agents" WHO said in a statement putting the death toll at at least 70. The United States has said the deaths were caused by sarin nerve gas dropped by Syrian aircraft. Russia has said it believes poison gas had leaked from a rebel chemical weapons depot struck by Syrian bombs. Sarin is an organophosporus compound and a nerve agent. Chlorine and mustard gas which are also believed to have been used in the past in Syria are not. A Russian Defence Ministry spokesman did not say what agent was used in the attack but said the rebels had used the same chemical weapons in Aleppo last year. The WHO said it was likely 

In [ ]:
print(train['statement'][0])
print(train['statement'][7])

wed apr 2017 syria attack symptom consist nerv agent use victim suspect chemic attack syria appear show symptom consist reaction nerv agent world health organ said wednesday case appear show addit sign consist exposur organophosphorus chemic categori chemic includ nerv agent said statement put death toll least unit state said death caus sarin nerv gas drop syrian aircraft russia said believ poison gas leak rebel chemic weapon depot struck syrian bomb sarin organophosporus compound nerv agent chlorin mustard gas also believ use past syria russian defenc ministri spokesman say agent use attack said rebel use chemic weapon aleppo last year said like kind chemic use attack becaus suffer appar extern injuri die rapid onset similar symptom includ acut respiratori distress said expert turkey give guidanc overwhelm health worker idlib diagnosi treatment patient medicin atropin antidot type chemic exposur steroid symptomat treatment sent commiss inquiri human right syria previous said forc loya

Nao apliquei stemm e depois lemm porque tira o sentido da palavra:  
*   train['statement'][7] ->onli person stage work activ last year pass along 
*   train_copy['statement'][7] -> on person stage work activ last year pas along rus feingold toughest ethic reform sinc waterg
russ feingold toughest ethic reform sinc waterg 

In [ ]:
#https://pt.stackoverflow.com/questions/407447/somar-os-valores-entre-os-dict
train_to_count=train.copy()
val_to_count=val.copy()
test_to_count=test.copy()

print(train_to_count['statement'][7])

sun feb 2015 suicid bomber kill assad clan hometown suicid bomber drive ambul kill four peopl sunday unpreced attack hospit took syria civil war rule assad clan hometown first time monitor group said attack came syrian observatori human right report troop execut peopl earlier week northern villag among children man drove ambul pack explos park qardaha hospit anoth man vehicl unclear whether accomplic hostag four peopl kill attack said observatori director rami abdel rahman earlier state televis report blast specifi natur attack attack first explos hit heart western town sinc outbreak syria civil war 2011 kill nurs hospit employe two soldier said observatori director abdel rahman outskirt qardaha previous come rocket fire latakia provinc town locat seen sever round heavi fight mausoleum contain grave presid bashar assad father predecessor hafez brother bassil locat qardaha clan rule syria iron fist year syria war began march 2011 prodemocraci revolt seek assad ouster morph conflict regi

In [ ]:
def CountOnly(sentence): #funçao para contar os valores unicos
  sentence=sentence.split()
  result = Counter(sentence)
  return result

train_to_count['statement']=train_to_count['statement'].apply(CountOnly) #aplicando a funçao ao dataset
val_to_count['statement']=val_to_count['statement'].apply(CountOnly)
test_to_count['statement']=test_to_count['statement'].apply(CountOnly)

c_train=Counter() #calculando a soma para o dataset train
c_val=Counter() #calculando a soma para o dataset val
c_test=Counter() #calculando a soma para o dataset test

print(train_to_count['statement'][7])

Counter({'kill': 13, 'peopl': 10, 'rebel': 9, 'observatori': 8, 'said': 7, 'area': 7, 'syria': 6, 'children': 6, 'attack': 5, 'assad': 4, 'troop': 4, 'villag': 4, 'two': 4, 'year': 4, 'clan': 3, 'hospit': 3, 'war': 3, 'report': 3, 'execut': 3, 'earlier': 3, 'week': 3, 'among': 3, 'qardaha': 3, 'abdel': 3, 'rahman': 3, 'town': 3, 'fire': 3, 'forc': 3, 'offens': 3, 'aleppo': 3, 'includ': 3, 'air': 3, 'citi': 3, 'also': 3, 'suicid': 2, 'bomber': 2, 'hometown': 2, 'ambul': 2, 'four': 2, 'took': 2, 'civil': 2, 'rule': 2, 'first': 2, 'group': 2, 'northern': 2, 'man': 2, 'explos': 2, 'director': 2, 'hit': 2, 'sinc': 2, '2011': 2, 'locat': 2, 'heavi': 2, 'fight': 2, 'meanwhil': 2, 'govern': 2, 'rityan': 2, 'six': 2, 'famili': 2, 'one': 2, 'part': 2, 'armi': 2, 'east': 2, 'alnusra': 2, 'accord': 2, 'raid': 2, 'saturday': 2, 'women': 2, 'barrel': 2, 'bomb': 2, 'regimeheld': 2, 'resolut': 2, 'order': 2, 'sieg': 2, 'past': 2, 'sun': 1, 'feb': 1, '2015': 1, 'drive': 1, 'sunday': 1, 'unpreced': 1, '

In [ ]:
for sentence in train_to_count['statement']:
  c_train = c_train + sentence 
for sentence in val_to_count['statement']:
  c_val = c_val + sentence 
for sentence in test_to_count['statement']:
  c_test = c_test + sentence 

list_unique_train=[] #criando lista pra selecionar os valores que aparecem uma unica vez no corpus
for k,v in c_train.items():
  if v==1:
    list_unique_train.append(k)
list_unique_val=[] 
for k,v in c_val.items():
  if v==1:
    list_unique_val.append(k)
list_unique_test=[] 
for k,v in c_test.items():
  if v==1:
    list_unique_test.append(k)

total_train=0
for k,v in c_train.items():
  total_train = total_train + v
total_test=0
for k,v in c_test.items():
  total_test = total_test + v
total_val=0
for k,v in c_val.items():
  total_val = total_val + v
print(total_train, total_val, total_test) 
print(len(list_unique_train),len(list_unique_val),len(list_unique_test))

102097 23191 27737
2809 1380 1682


In [ ]:
print("0 conjunto de treino atual final correspondeu a aproximadamente: ", round(((total_train-len(list_unique_train))/total_train)*100,1), "% do conjunto original com um total de: ",(total_train-len(list_unique_train)),"palavras")
print("0 conjunto de teste atual final correspondeu a aproximadamente: ", round(((total_test-len(list_unique_test))/total_test)*100,1), "% do conjunto original com um total de: ",(total_test-len(list_unique_test)),"palavras")
print("0 conjunto de validaçao atual  final correspondeu a aproximadamente: ", round(((total_val-len(list_unique_val))/total_val)*100,1), "% do conjunto original com um total de: ",(total_val-len(list_unique_val)),"palavras")

0 conjunto de treino atual final correspondeu a aproximadamente:  97.2 % do conjunto original com um total de:  99288 palavras
0 conjunto de teste atual final correspondeu a aproximadamente:  93.9 % do conjunto original com um total de:  26055 palavras
0 conjunto de validaçao atual  final correspondeu a aproximadamente:  94.0 % do conjunto original com um total de:  21811 palavras


In [ ]:
##https://pt.stackoverflow.com/questions/407447/somar-os-valores-entre-os-dict
def cleanningOnceTrain(statement): #criando funçao para limpar do corpus aquelas palavras que ocorrem uma unica vez
#  for item in [train]: #tiro apenas desse conjunto porque por mais que em outro ocorra uma unica vez, nesse conjunto pode ser que aquela palavra seja relevante
  statement=statement.split()
  result = [word for word in statement if word not in list_unique_train]
  result = " ".join(result)
  return result
def cleanningOnceVal(statement): #criando funçao para limpar do corpus aquelas palavras que ocorrem uma unica vez
#  for item in [val]:
  statement=statement.split()
  result = [word for word in statement if word not in list_unique_val]
  result = " ".join(result)
  return result
def cleanningOnceTest(statement): #criando funçao para limpar do corpus aquelas palavras que ocorrem uma unica vez
  statement=statement.split()
  result = [word for word in statement if word not in list_unique_test]
  result = " ".join(result)
  return result

In [ ]:
train_copy=train.copy()
test_copy=test.copy()
val_copy=val.copy()

train_copy['statement']=train_copy['statement'].apply(cleanningOnceTrain) #aplicando ao conjunto de dados
val_copy['statement']=val_copy['statement'].apply(cleanningOnceVal)
test_copy['statement']=test_copy['statement'].apply(cleanningOnceTest)

In [ ]:
train_copy['statement'][0]

'wed apr 2017 syria attack symptom consist nerv agent use victim suspect chemic attack syria appear show symptom consist reaction nerv agent world health organ said wednesday case appear show addit sign consist exposur chemic categori chemic includ nerv agent said statement put death toll least unit state said death caus sarin nerv gas drop syrian aircraft russia said believ poison gas rebel chemic weapon depot struck syrian bomb sarin compound nerv agent chlorin mustard gas also believ use past syria russian defenc ministri spokesman say agent use attack said rebel use chemic weapon aleppo last year said like kind chemic use attack becaus suffer appar extern injuri die rapid similar symptom includ acut respiratori distress said expert turkey give overwhelm health worker idlib treatment patient medicin type chemic exposur treatment sent commiss inquiri human right syria previous said forc loyal syrian presid bashar alassad use lethal chlorin gas multipl hundr civilian die sarin gas att

In [ ]:
train['statement']=train['statement'].apply(cleanningOnceTrain) #aplicando ao conjunto de dados
val['statement']=val['statement'].apply(cleanningOnceVal)
test['statement']=test['statement'].apply(cleanningOnceTest)

### Case 1 (c) : Pre process 


https://huggingface.co/datasets/liar, https://arxiv.org/pdf/2009.01047v2.pdf 

In [ ]:
%%time 
#0 fake 1 true 
X_train_c = train['statement'] 
X_val_c = val['statement']
X_test_c = test['statement']
y_train_c = train['label']
y_val_c = val['label']
y_test_c = test['label']

print(y_train_c.count())
print(y_train_c.value_counts())
print(y_train_c.value_counts(1))

train_c=train.copy()
test_c=test.copy()
val_c=val.copy()


514
1    284
0    230
Name: label, dtype: int64
1    0.552529
0    0.447471
Name: label, dtype: float64
CPU times: user 37.3 ms, sys: 0 ns, total: 37.3 ms
Wall time: 42 ms


In [ ]:
train_c.to_csv('/content/train_2c.csv')
test_c.to_csv('/content/test_2c.csv')
val_c.to_csv('/content/val_2c.csv')

### Embeddings 

In [ ]:
#Features
w2v = EmbeddingTransformer('word2vec')

[=================================================-] 99.4% 1653.1/1662.8MB downloaded


In [ ]:
w2v.fit_transform(X_train_c)

array([[-0.00273969,  0.02318632,  0.05321609, ..., -0.02540818,
         0.03828512,  0.04682676],
       [ 0.00945816,  0.02785452,  0.03692653, ..., -0.04043502,
        -0.0155724 ,  0.02396633],
       [-0.00483964,  0.03137188,  0.02934786, ..., -0.04806853,
         0.02535862,  0.00162711],
       ...,
       [-0.00808173,  0.01673709,  0.06431025, ..., -0.02560842,
        -0.00456041,  0.03682247],
       [ 0.01025352,  0.04365415,  0.0704158 , ..., -0.02342033,
         0.03085198,  0.04366211],
       [-0.05440822, -0.03893165,  0.0665986 , ..., -0.01848995,
         0.00662879, -0.00485692]], dtype=float32)

In [ ]:
X_train_c[0]

'wed apr 2017 syria attack symptom consist nerv agent use victim suspect chemic attack syria appear show symptom consist reaction nerv agent world health organ said wednesday case appear show addit sign consist exposur chemic categori chemic includ nerv agent said statement put death toll least unit state said death caus sarin nerv gas drop syrian aircraft russia said believ poison gas rebel chemic weapon depot struck syrian bomb sarin compound nerv agent chlorin mustard gas also believ use past syria russian defenc ministri spokesman say agent use attack said rebel use chemic weapon aleppo last year said like kind chemic use attack becaus suffer appar extern injuri die rapid similar symptom includ acut respiratori distress said expert turkey give overwhelm health worker idlib treatment patient medicin type chemic exposur treatment sent commiss inquiri human right syria previous said forc loyal syrian presid bashar alassad use lethal chlorin gas multipl hundr civilian die sarin gas att

In [ ]:
w2v.fit_transform(X_train_c[0])

array([[-0.23339844,  0.06152344, -0.3046875 , ..., -0.18359375,
        -0.15039062,  0.10986328],
       [-0.06933594,  0.15332031, -0.02490234, ...,  0.06054688,
        -0.19238281,  0.27148438],
       [-0.09082031,  0.25585938,  0.08935547, ..., -0.13476562,
        -0.1484375 ,  0.05151367],
       ...,
       [-0.33789062,  0.19824219, -0.296875  , ..., -0.15917969,
         0.03417969,  0.09179688],
       [-0.06933594,  0.15332031, -0.02490234, ...,  0.06054688,
        -0.19238281,  0.27148438],
       [-0.30078125,  0.18945312, -0.03491211, ..., -0.10009766,
        -0.12060547,  0.2890625 ]], dtype=float32)

In [ ]:
w2v.fit_transform(X_train_c[0]).shape

(1379, 300)

In [ ]:
w2v.fit_transform(X_train_c[7]).shape

(2471, 300)

In [ ]:
glove = EmbeddingTransformer('glove')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
glove.fit_transform(X_train_c)

array([[-0.01221257,  0.36043975, -0.40091667, ..., -0.135212  ,
        -0.9389165 , -0.22925848],
       [-0.00335899,  0.33640224, -0.20052586, ..., -0.10878191,
        -0.6621473 , -0.235027  ],
       [-0.01492438,  0.29794568, -0.02607873, ..., -0.04902851,
        -0.83119345, -0.00309012],
       ...,
       [-0.0404934 ,  0.38074598, -0.45183033, ..., -0.09911146,
        -0.6090198 , -0.35131854],
       [ 0.08261196,  0.40270796, -0.39557984, ..., -0.20566496,
        -0.83938515, -0.229723  ],
       [-0.02153397,  0.30540136, -0.28026855, ...,  0.05284717,
        -0.9823406 , -0.07080006]], dtype=float32)

In [ ]:
glove.fit_transform(X_train_c[0]).shape

(1379, 25)

In [ ]:
glove.fit_transform(X_train_c[7]).shape

(2471, 25)

In [ ]:
fasttext = EmbeddingTransformer('fasttext')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
fasttext.fit_transform(X_train_c)

array([[ 0.00905007, -0.01172359, -0.00142608, ...,  0.02316247,
        -0.00886377, -0.01573206],
       [ 0.0032581 , -0.01445853,  0.0075    , ...,  0.00221557,
         0.00279631, -0.0076632 ],
       [-0.00699969, -0.00749606,  0.0137572 , ..., -0.00558687,
         0.01953606,  0.01521506],
       ...,
       [ 0.01380103, -0.01280375, -0.00024101, ..., -0.00497443,
         0.00257933, -0.00556467],
       [ 0.0033687 , -0.01208537,  0.00822072, ..., -0.00414088,
        -0.00130406, -0.00679279],
       [-0.003023  , -0.00947885,  0.0041557 , ..., -0.00686648,
         0.00991748, -0.0064911 ]], dtype=float32)

In [ ]:
fasttext.fit_transform(X_train_c[0]).shape

(1379, 300)

In [ ]:
fasttext.fit_transform(X_train_c[7]).shape

(2471, 300)

In [ ]:
#cv = CountVectorizer(analyzer='word', tokenizer= Tokenizer, lowercase=True, stop_words=stopwords_list)
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
#tfidf =  TfidfVectorizer(analyzer='word',  tokenizer= Tokenizer, lowercase=True, use_idf=True, stop_words=stopwords_list)
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')

In [ ]:
cv.fit_transform(X_train_c)
tfidf.fit_transform(X_train_c)

<514x3732 sparse matrix of type '<class 'numpy.float64'>'
	with 57224 stored elements in Compressed Sparse Row format>

In [ ]:
#Features
#import tensorflow
#with tensorflow.device('/gpu:0'):
#  cv = CountVectorizer(analyzer='word', tokenizer= Tokenizer, lowercase=True, stop_words=stopwords_list)
#  tfidf =  TfidfVectorizer(analyzer='word',  tokenizer= Tokenizer, lowercase=True, use_idf=True, stop_words=stopwords_list)
#  w2v = EmbeddingTransformer('word2vec')
#  glove = EmbeddingTransformer('glove')
#  fasttext = EmbeddingTransformer('fasttext')


In [ ]:
#!pip install tokenizerBert

In [ ]:
#!pip install tokenizerBert
#import tokenizerBert
#def transformBert(text):
#	encoded_input = tokenizerBert.encode(text, return_tensors='tf', add_special_tokens=True, max_lenght=512)
#	return modelBert(encoded_input)[0][0][0]
#def transformBertList(list):
#	return numpy.asarray([transformBert(text) for text in list])
#def LoadBertEmbedding():
#	return FunctionTransformer(labda x: transformBertList(x), validate=False)

## PARTE II

### Classificadores

In [ ]:
#--Classifiers
#--SVM
def get_svm(cv, tfidf, w2v, glove, fasttext):
    return{
        'CV': {'CLF': SVC(random_state=42, kernel='linear', gamma=0.1, probability=True),'EXT': cv,},
        'TFIDF': {'CLF': SVC(random_state=42, kernel='linear',
                             gamma=0.1, probability=True),'EXT': tfidf,},
        'W2V': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': w2v,},
        'GLOVE': {'CLF': SVC(random_state=42, kernel='rbf', gamma=0.5, probability=True),'EXT':  glove,},
        'FAST': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': fasttext,}
    }
#--LR
def get_lr(cv,tfidf,glove,w2v,fasttext):
    return {
        'CV': {'CLF': LogisticRegression(random_state=42,multi_class='auto', solver='liblinear', penalty='l1'),'EXT': cv,},
        'TFIDF': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': tfidf,},
        'GLOVE': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT':  glove,},
        'W2V': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': w2v,},
        'FAST': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': fasttext,}
        }
#--RF
def get_rf(cv, tfidf, w2v, glove, fasttext):
    return {'CV': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=20, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
            }
#--NB
def get_nb(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': MultinomialNB(alpha=1, fit_prior=False),'EXT': cv,},
        'TFIDF': {'CLF': MultinomialNB(alpha=0.5, fit_prior=False),'EXT': tfidf,},
        'W2V': {'CLF': BernoulliNB(alpha=0.5, fit_prior=True),'EXT': w2v,},
        'GLOVE': {'CLF': BernoulliNB(alpha=0.1, fit_prior=True),'EXT':  glove,},
        'FAST': {'CLF': BernoulliNB(alpha=1, fit_prior=True),'EXT': fasttext,}
    }
#--MLP
def get_mlp(cv, tfidf, w2v, glove, fasttext):
    return {'CV': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='relu', solver='lbfgs'),'EXT': cv,},
        'TFIDF': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='logistic', solver='adam'),'EXT': tfidf,},
        'W2V': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': w2v,},
        'GLOVE': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT':  glove,},
        'FAST': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': fasttext,}
    }
#--EXTRA
def get_extra(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
    }
#--KNN
def get_knn(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': KNeighborsClassifier(n_neighbors=3, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': fasttext,}
    }

def get_CNN_2(ext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, MAX_SEQUENCE_LENGTH=300, activation='sigmoid', word_embedding=False, dense=2):
    if word_embedding == False:
        X_ext = ext.get_feature_names()
        model = Word2Vec([X_ext], min_count=1, workers=1, size=300)
    else:
        model = ext.model

    word_index = tokenizer.word_index

    nb_words = min(MAX_NB_WORDS, len(word_index)) + 1

    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if word in model.wv.vocab:
            embedding_matrix[i] = model.wv.word_vec(word)

    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False
                                )
    cnn = Sequential()
    cnn.add(embedding_layer)
    cnn.add(Dropout(0.2))
    cnn.add(Conv1D(64, 2, padding='valid', activation='relu', strides=1))
    cnn.add(GlobalMaxPooling1D())
    cnn.add(Dense(256))
    cnn.add(Dropout(0.2))
    cnn.add(Activation('relu'))
    cnn.add(Dense(2))
    cnn.add(Activation(activation))
    cnn.summary()
    cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return cnn

In [ ]:
def get_classifier(clf, X, y, ext):
    pipe_clf = Pipeline([('extractor', ext),('clf', clf)])
    pipe_clf.fit(X, y)
    return pipe_clf

In [ ]:
def update_pred_proba_2(clfs, label, train, val, test):
    pred_train = pd.DataFrame()
    pred_val = pd.DataFrame()
    pred_test = pd.DataFrame()

    prob_train = pd.DataFrame()
    prob_val = pd.DataFrame()
    prob_test = pd.DataFrame()

    for ext, clf in clfs.items():
        # Predict
        df_pred_train_ = pd.DataFrame(clf.predict(train), columns=["{}-{}".format(label, ext)])
        df_pred_val_ = pd.DataFrame(clf.predict(val), columns=["{}-{}".format(label, ext)])
        df_pred_test_ = pd.DataFrame(clf.predict(test), columns=["{}-{}".format(label, ext)])
        
        # # Probabilidades
        cols = [
            "{}-{}-{}".format(label, ext, clf.classes_[0]), 
            "{}-{}-{}".format(label, ext, clf.classes_[1])
        ]
        df_prob_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
        df_prob_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
        df_prob_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)

        pred_train = pd.concat([pred_train, df_pred_train_], axis=1, sort=False)
        pred_val = pd.concat([pred_val, df_pred_val_], axis=1, sort=False)
        pred_test = pd.concat([pred_test, df_pred_test_], axis=1, sort=False)

        prob_train = pd.concat([prob_train, df_prob_train_], axis=1, sort=False)
        prob_val = pd.concat([prob_val, df_prob_val_], axis=1, sort=False)
        prob_test = pd.concat([prob_test, df_prob_test_], axis=1, sort=False)

    return  pred_train, pred_val, pred_test, prob_train, prob_val, prob_test

In [ ]:
!mkdir Data
!mkdir Data/fakes

#### Caso 2 (c) - Binary 

In [ ]:
del train, test, val

In [ ]:
#--data
train=pd.DataFrame(train_c[['statement','label']])
train=train.reset_index()   #pd.DataFrame(dataset['train'])
train=train.drop(columns=['index'])
test=pd.DataFrame(test_c[['statement','label']])#dataset['test'])
test=test.reset_index()
test=test.drop(columns=['index'])
val=pd.DataFrame(val_c[['statement','label']])#dataset['validation'])
val=val.reset_index()
val=val.drop(columns=['index'])
# load data
train_2_c=train
train.to_csv('/content/Data/fakes/train_2_c.csv')
test_2_c=test
test.to_csv('/content/Data/fakes/test_2_c.csv')
val_2_c=val
val.to_csv('/content/Data/fakes/val_2_c.csv')
y_train=pd.DataFrame(train['label'])#train['label']
y_train.to_csv('/content/Data/fakes/train_labels_2_c.csv')
y_test=pd.DataFrame(test['label'])#test['label']
y_test.to_csv('/content/Data/fakes/test_labels_2_c.csv')
y_val=pd.DataFrame(val['label'])#val['label']
y_val.to_csv('/content/Data/fakes/val_labels_2_c.csv')

In [ ]:
X_train_c=train['statement']
y_train_c=train['label']
X_val_c=val['statement']
y_val_c=val['label']
X_test_c=test['statement']
y_test_c=test['label']

In [ ]:
print(pd.isnull(train['label']).sum())
print(pd.isnull(test['label']).sum())
print(pd.isnull(val['label']).sum())

0
0
0


In [ ]:
print(pd.isnull(y_train).sum(),pd.isnull(y_val).sum(),pd.isnull(y_test).sum())
print(y_train.shape,y_val.shape,y_test.shape)

label    0
dtype: int64 label    0
dtype: int64 label    0
dtype: int64
(514, 1) (129, 1) (161, 1)


In [ ]:
%%time
def main_2_c():
  for db in ["fakes"]:
    if db == 'fakes':
      classe = 'label'
      df_pred_train = pd.DataFrame()# inicializando pred 
      df_pred_val = pd.DataFrame()
      df_pred_test = pd.DataFrame()
      df_prob_val = pd.DataFrame()# inicializando proba
      df_prob_train = pd.DataFrame()
      df_prob_test = pd.DataFrame()
      
      df_pred_train = pd.concat([df_pred_train, train_2_c['label']], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, val_2_c['label']], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, test_2_c['label']], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, train_2_c['label']], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, val_2_c['label']], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, test_2_c['label']], axis=1, sort=False)

      print('dados ok')

      # SVM
      classifier = get_svm(cv, tfidf, w2v, glove, fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "SVM", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "SVM", X_train_c, X_val_c, X_test_c)

      print('svm parte 1 ok')
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('svm parte 2 ok')

      # LR
      classifier = get_lr(cv,tfidf,glove,w2v,fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])

      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "LR", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "LR", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('lr ok')

      # RF
      classifier = get_rf(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "RF", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "RF", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('rf ok')

      # NB
      classifier = get_nb(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "NB", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "NB", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('nb ok')

      # MLP
      classifier = get_mlp(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "MLP", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "MLP", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('mlp ok')

      # Extra
      classifier = get_extra(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "EXTRA", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "EXTRA", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('extra ok')

      # KNN
      classifier = get_knn(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "KNN", X_train_c, X_val_c, X_test_c)
      #pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "KNN", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('knn ok')

      #CNN
      MAX_NB_WORDS = 20000
      MAX_SEQUENCE_LENGTH=300
      tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
      tokenizer.fit_on_texts(X_train_c) #rever por que 
      seq_train = tokenizer.texts_to_sequences(X_train_c)
      seq_val = tokenizer.texts_to_sequences(X_val_c)
      seq_test = tokenizer.texts_to_sequences(X_test_c)

      data_train = pad_sequences(seq_train,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_val = pad_sequences(seq_val,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_test = pad_sequences(seq_test, padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
                            
      # NET
      cnn_cv = get_CNN_2(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=2)
      cnn_tfidf = get_CNN_2(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=2)
      cnn_w2v = get_CNN_2(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
      cnn_glove = get_CNN_2(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=2)
      cnn_fast = get_CNN_2(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
      
      y_validation=to_categorical(y_val_c)
      y_training=to_categorical(y_train_c)
      y_testing=to_categorical(y_test_c)
      
      # TRAIN
      cnn_cv.fit(data_train, y_training, validation_data=(data_val,y_validation), epochs=20, batch_size=20)
      cnn_tfidf.fit(data_train, y_training, validation_data=(data_val,y_validation), epochs=20, batch_size=20)
      cnn_w2v.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_glove.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_fast.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      
       #predictions and proba
      cols = ["CNN-CV-0", "CNN-CV-1"]
      
      df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
      df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
      df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)
      
      cols = ["CNN-TF-0", "CNN-TF-1"]
      df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
      df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
      df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)
      
      cols = ["CNN-W2V-0", "CNN-W2V-1"]
      df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
      df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
      df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-GLOVE-0", "CNN-GLOVE-1"]
      df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
      df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
      df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-FAST-0", "CNN-FAST-1"]
      df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
      df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
      df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)       

      print('cnn ok')     

      # Probabilidade
      path = "./Data/{}".format(db)
      df_pred_train.to_csv("{}/pred_train_2_c.csv".format(path))
      df_pred_val.to_csv("{}/pred_val_2_c.csv".format(path))
      df_pred_test.to_csv("{}/pred_test_2_c.csv".format(path))

      df_prob_train.to_csv("{}/prob_train_2_c.csv".format(path))
      df_prob_val.to_csv("{}/prob_val_2_c.csv".format(path))
      df_prob_test.to_csv("{}/prob_test_2_c.csv".format(path))

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 12.4 µs


In [ ]:
%%time
def main_2_c():
  for db in ["fakes"]:
    if db == 'fakes':
      classe = 'label'

      df_pred_val = pd.DataFrame()
      df_prob_val = pd.DataFrame()# inicializando proba

      df_pred_val = pd.concat([df_pred_val, val_2_c['label']], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, val_2_c['label']], axis=1, sort=False)
      print('dados ok')

      # SVM
      classifier = get_svm(cv, tfidf, w2v, glove, fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])
      pred_val, prob_val = update_pred_proba_2(clfs, "SVM", X_val_c)
      print('svm parte 1 ok')
      
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      print('svm parte 2 ok')

      
      # Probabilidade
      path = "./Data/{}".format(db)
      df_pred_val.to_csv("{}/pred_val_2_c.csv".format(path))
      df_prob_val.to_csv("{}/prob_val_2_c.csv".format(path))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.78 µs


In [ ]:
%%time 
if __name__ == "__main__":
    main_2_c()

dados ok
svm parte 1 ok
svm parte 2 ok
lr ok
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
building tree 4 of 20
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
building tree 5 of 20[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s

building tree 6 of 20[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s

building tree 7 of 20[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s

building tree 8 of 20[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s

building tree 9 of 20
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.0s
building tree 10 of 20[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
building tree 11 of 20

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
building tree 12 of 20[Parallel(n_jobs=-1)]: Done  10 tasks      | elaps

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 300, 300)          1148100   
                                                                 
 dropout_30 (Dropout)        (None, 300, 300)          0         
                                                                 
 conv1d_15 (Conv1D)          (None, 299, 64)           38464     
                                                                 
 global_max_pooling1d_15 (Gl  (None, 64)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_30 (Dense)            (None, 256)               16640     
                                                                 
 dropout_31 (Dropout)        (None, 256)               0         
                                                     

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 300, 300)          1148100   
                                                                 
 dropout_32 (Dropout)        (None, 300, 300)          0         
                                                                 
 conv1d_16 (Conv1D)          (None, 299, 64)           38464     
                                                                 
 global_max_pooling1d_16 (Gl  (None, 64)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_32 (Dense)            (None, 256)               16640     
                                                                 
 dropout_33 (Dropout)        (None, 256)               0         
                                                     

In [ ]:
!cp -r "/content/Data/fakes/pred_test_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/pred_test_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/pred_val_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/prob_test_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/prob_val_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/test_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/test_labels_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/train_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/test_labels_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/val_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/val_labels_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"
!cp -r "/content/Data/fakes/train_labels_2_c.csv" "/content/drive/MyDrive/cin_Pesquisa/NLP:FakeNews/4_Codes/FN/Analysis Clusters/files_casos_1_2_3_4_revisando/fakes/2/"